# Project Submission

This notebook will be your project submission. All tasks will be listed in the order of the Courses that they appear in. The tasks will be the same as in the Capstone Example Notebook, but in this submission you ___MUST___ use another dataset. Failure to do so will result in a large penalty to your grade in this course.

## Finding your dataset

Take some time to find an interesting dataset! There is a reading discussing various places where datasets can be found, but if you are able to process it, go ahead and use it! Do note, for some tasks in this project, each entry will need 3+ attributes, so keep that in mind when finding datasets. After you have found your dataset, the tasks will continue as in the Example Notebook. You will be graded based on the tasks and your results. Best of luck!

### As Reviewer: 
Your job will be to verify the calculations made at each "TODO" labeled throughout the notebook.

### First Step: Imports

In the next cell we will give you all of the imports you should need to do your project. Feel free to add more if you would like, but these should be sufficient.

In [1]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize
import string
from sklearn import linear_model
from nltk.stem.porter import PorterStemmer # Stemming

# Task 1: Data Processing

### TODO 1: Read the data and Fill your dataset

In [2]:
#YOUR CODE HERE
path = 'amazon_reviews_us_Gift_Card_v1_00.tsv.gz'
f = gzip.open(path, 'rt', encoding='utf8')

In [3]:
header = f.readline()
header = header.strip().split('\t')

In [4]:
dataset = []
for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header,fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    d['verified_purchase'] = d['verified_purchase'] == 'Y'
    dataset.append(d)

### TODO 2: Split the data into a Training and Testing set

First shuffle your data, then split your data. Have Training be the first 80%, and testing be the remaining 20%. 

In [5]:
#YOUR CODE HERE
random.shuffle(dataset)

N = len(dataset)

trainingSet = dataset[:8*N//10]
testSet = dataset[8*N//10:]

#### Now delete your dataset
You don't want any of your answers to come from your original dataset any longer, but rather your Training Set, this will help you to not make any mistakes later on, especialy when referencing the checkpoint solutions.

In [6]:
#YOUR CODE HERE
del dataset

### TODO 3: Extracting Basic Statistics

Next you need to answer some questions through any means (i.e. write a function or just find the answer) all based on the __Training Set:__
1. How many entries are in your dataset?
2. Pick a non-trivial attribute (i.e. verified purchases in example), what percentage of your data has this atttribute?
3. Pick another different non-trivial attribute, what percentage of your data share both attributes?

# Task 2: Classification

Next you will use our knowledge of classification to extract features and make predictions based on them. Here you will be using a Logistic Regression Model, keep this in mind so you know where to get help from.

### TODO 1: Define the feature function

This implementation will be based on ___any two___ attributes from your dataset. You will be using these two attributes to predict a third. Hint: Remember the offset!

In [7]:
#FIX THIS

def feature(d):
    feat = [1, d['star_rating'], len(d['review_body'])]
    return feat

### TODO 2: Fit your model

1. Create your __Feature Vector__ based on your feature function defined above. 
2. Create your __Label Vector__ based on the "verified purchase" column of your training set.
3. Define your model as a __Logistic Regression__ model.
4. Fit your model.

In [8]:
#YOUR CODE HERE
X = [feature(d) for d in trainingSet]
y = [d['star_rating'] for d in trainingSet]

In [9]:
model = linear_model.LogisticRegression()

In [10]:
y_class = [(rating > 2) for rating in y]
model.fit(X, y_class)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### TODO 3: Compute Accuracy of Your Model

1. Make __Predictions__ based on your model.
2. Compute the __Accuracy__ of your model.

In [11]:
#YOUR CODE HERE
predictions = model.predict(X)

In [12]:
correct = predictions == y_class

In [13]:
accuracy = sum(correct) / len(correct)
print('Accuracy = ' + str(accuracy))

Accuracy = 1.0


# Task 3: Regression

In this section you will start by working though two examples of altering features to further differentiate. Then you will work through how to evaluate a Regularaized model.

In [14]:
#CHANGE PATH
path = 'amazon_reviews_us_Gift_Card_v1_00.tsv.gz'

#GIVEN
f = gzip.open(path, 'rt', encoding="utf8")
header = f.readline()
header = header.strip().split('\t')
reg_dataset = []
for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    reg_dataset.append(d)

### TODO 1: Unique Words in a Sample Set

We are going to work with a new dataset here, as such we are going to take a smaller portion of the set and call it a Sample Set. This is because stemming on the normal training set will take a very long time. (Feel free to change sampleSet -> reg_dataset if you would like to see the difference for yourself)

1. Count the number of unique words found within the 'review body' portion of the sample set defined below, making sure to __Ignore Punctuation and Capitalization__.
2. Count the number of unique words found within the 'review body' portion of the sample set defined below, this time with use of __Stemming,__ __Ignoring Puctuation,__ ___and___ __Capitalization__.

In [15]:
#GIVEN for 1.
wordCount = defaultdict(int)
punctuation = set(string.punctuation)

#GIVEN for 2.
wordCountStem = defaultdict(int)
stemmer = PorterStemmer() #use stemmer.stem(stuff)

#SampleSet and y vector given
sampleSet = reg_dataset[:2*len(reg_dataset)//10]
y_reg = [d['star_rating'] for d in sampleSet]

In [16]:
#Ignoring punctuation and capitalization
for d in sampleSet:
    r = ''.join([c for c in d['review_body'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

print(len(wordCount))

10765


In [17]:
#Ignoring punctuation, capitalization, and stemming
for d in sampleSet:
    r = ''.join([c for c in d['review_body'].lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w) # with stemming
        wordCountStem[w] += 1

print(len(wordCountStem))

8389


### TODO 2: Evaluating Classifiers

1. Given the feature function and your counts vector, __Define__ your X_reg vector. (This being the X vector, simply labeled for the Regression model)
2. __Fit__ your model using a __Ridge Model__ with (alpha = 1.0, fit_intercept = True).
3. Using your model, __Make your Predictions__.
4. Find the __MSE__ between your predictions and your y_reg vector.

In [18]:
#GIVEN FUNCTIONS
def feature_reg(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_body'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    return feat

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

#GIVEN COUNTS AND SETS
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

#Note: increasing the size of the dictionary may require a lot of memory
words = [x[1] for x in counts[:100]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [19]:
#YOUR CODE HERE
X_reg = [feature(d) for d in sampleSet]

In [20]:
model = linear_model.Ridge(1.0, fit_intercept=True)
model.fit(X_reg,y_reg)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
predictions = model.predict(X_reg)

In [22]:
differences = [(x-y)**2 for (x,y) in zip(predictions, y_reg)]

MSE = sum(differences) / len(differences) # mean square error
print('MSE = ' + str(MSE))

MSE = 1.907724670453836e-09


# Task 4: Recommendation Systems

For your final task, you will use your knowledge of simple similarity-based recommender systems to make calculate the most similar items.

The next cell contains some starter code that you will need for your tasks in this section.
Notice you should be back to using your __trainingSet__.

In [23]:
#GIVEN
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

### TODO 1: Fill your Dictionaries

1. For each entry in your training set, fill your default dictionaries (defined above). 

In [24]:
#YOUR CODE HERE
for t in trainingSet:
    user,item = t['customer_id'], t['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)

In [25]:
#GIVEN
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

def mostSimilar(n, m): #n is the entry index
    similarities = []  #m is the number of entries
    users = usersPerItem[n]
    for i2 in usersPerItem:
        if i2 == n: continue
        sim = Jaccard(users, usersPerItem[n])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:m]

### TODO 1: Fill your Dictionaries

1. Calculate the __10__ most similar entries to the __first__ entry in your dataset, using the functions defined above.

In [26]:
#YOUR CODE HERE
query = trainingSet[1]['product_id']
mostSimilar(query, 10)

[(1.0, 'BT00DDZD6G'),
 (1.0, 'BT00DDVMVQ'),
 (1.0, 'BT00DDC88W'),
 (1.0, 'BT00DDC7CE'),
 (1.0, 'BT00DDC7C4'),
 (1.0, 'BT00DDC7BK'),
 (1.0, 'BT00DDBSA6'),
 (1.0, 'BT00DDBS9M'),
 (1.0, 'BT00DC6QU4'),
 (1.0, 'BT00CTPKCS')]

## Finished!

Congratulations! You are now ready to submit your work. Once you have submitted make sure to get started on your peer reviews!